# Latent Dirichlet Allocation

<img src='https://miro.medium.com/max/1014/1*2uj6t3gNv76SpHrWf5-z-A.jpeg' img/>

<img src='https://miro.medium.com/max/2361/1*fCc0JT3W-1ViYyw0hJ7rdA.jpeg' img/>

### Resources: 

https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-latent-dirichlet-allocation-437c81220158

https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d

https://www.coursera.org/lecture/text-mining/3-9-latent-dirichlet-allocation-lda-part-1-deiXc

https://www.coursera.org/lecture/ml-clustering-and-retrieval/mixed-membership-models-for-documents-hQBJI

## Data

We will be using articles from NPR (National Public Radio), obtained from their website [www.npr.org](http://www.npr.org)

In [1]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 110kB/s eta 0:00:01
     |████████████████████████████████| 552kB 3.7MB/s eta 0:00:01
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=cc05b3bf5265aab6df7e46422e6de137092ff8b3620ada7aeadb2377f29ee7d9
  Stored in directory: /Users/schlinkertc/Library/Caches/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32040 sha256=e52419f24455f1946441354c52f0a183f468a4343b88270c63b63f4b957f27f4
  Stored in directory: /Users/schlinkertc/Library/Caches/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [42]:
import pandas as pd

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [5]:
npr = pd.read_csv('data/npr.csv')

In [6]:
npr.shape

(11992, 1)

In [7]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Preprocessing

In [8]:
import re 
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def spacy_tokenizer(text):
    # remove html tags from all of the text before processing
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    # Creating our token object, which is used to create documents with linguistic annotations.
    # we disabled the parser and ner parts of the pipeline in order to speed up parsing
    mytokens = nlp(cleantext, disable=['parser', 'ner'])

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [10]:
cv = CountVectorizer(tokenizer=spacy_tokenizer, max_df=0.90, min_df=10, stop_words='english')

In [11]:
dtm = cv.fit_transform(npr['Article'])

In [12]:
dtm

<11992x17335 sparse matrix of type '<class 'numpy.int64'>'
	with 2635570 stored elements in Compressed Sparse Row format>

## LDA

In [13]:
from sklearn.decomposition import LatentDirichletAllocation

In [15]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,           # Number of topics
                                      max_iter=20,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=20,
                          mean_change_tol=0.001, n_components=20, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [16]:
# This can take awhile, we're dealing with a large amount of documents!

lda_output = lda_model.fit_transform(dtm)


### Diagnose model performance with perplexity and log-likelihood
A model with higher log-likelihood and lower perplexity (exp(-1. * log-likelihood per word)) is considered to be good. Let’s check for our model.



In [19]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(dtm))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(dtm))

# See model parameters
print(lda_model.get_params())

Log Likelihood:  -33398970.374681056
Perplexity:  2550.37356879694
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'online', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 20, 'mean_change_tol': 0.001, 'n_components': 20, 'n_jobs': -1, 'perp_tol': 0.1, 'random_state': 100, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


## Showing Stored Words

In [20]:
len(cv.get_feature_names())

17335

In [21]:
import random

In [23]:
for i in range(10):
    random_word_id = random.randint(0,17335)
    print(cv.get_feature_names()[random_word_id])

volt
drain
fervent
picket
instill
civic
kenyan
rife
firecracker
sexualize


In [24]:
for i in range(10):
    random_word_id = random.randint(0,17335)
    print(cv.get_feature_names()[random_word_id])

mutant
dissolve
party
gilmore
kinship
navarro
mega
headset
ken
romance


### Showing Top Words Per Topic

In [25]:
len(lda_model.components_)

20

In [26]:
lda_model.components_

array([[3.00846407e+02, 4.66407437e+00, 5.00000037e-02, ...,
        1.93261495e+03, 5.00000010e-02, 3.45371006e+00],
       [1.75563323e+01, 4.46091330e+00, 1.96986483e+02, ...,
        2.96276380e+03, 5.00000008e-02, 1.45859257e+01],
       [1.40411340e+02, 5.00000027e-02, 5.00000048e-02, ...,
        2.48957604e+03, 4.35125447e+00, 5.00000050e-02],
       ...,
       [2.38978781e+03, 7.85161028e+01, 2.04439988e+01, ...,
        1.10369829e+04, 5.53495593e+01, 5.00000024e-02],
       [5.01021180e-02, 5.00000008e-02, 5.00000041e-02, ...,
        1.20385678e+03, 5.00000019e-02, 5.00000008e-02],
       [1.80098782e+02, 5.00000015e-02, 5.00000024e-02, ...,
        3.95613641e+03, 6.75677991e+00, 5.00000023e-02]])

In [27]:
len(lda_model.components_[0])

17335

In [28]:
single_topic = lda_model.components_[0]

In [29]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([ 3926,  5321,  7572, ..., 10894, 12189, 16014])

In [32]:
# Word least representative of this topic
single_topic[-1]

3.4537100634917515

In [33]:
# Word most representative of this topic
single_topic[0]

300.84640709677893

In [34]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([17020,   636, 17331, 13578, 16112, 13593,  7639, 10894, 12189,
       16014])

In [35]:
top_word_indices = single_topic.argsort()[-10:]

In [36]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

white
administration
—
russia
u.
s.
house
obama
president
trump


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [37]:
for index,topic in enumerate(lda_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['russian', 'statement', 'report', 'tell', '’', 'white', 'administration', '—', 'russia', 'u.', 's.', 'house', 'obama', 'president', 'trump']


THE TOP 15 WORDS FOR TOPIC #1
['tax', 'program', 'cost', 'money', 'million', '—', '’', 'plan', 'pay', 'people', 'care', 'state', 'year', 'health', 'percent']


THE TOP 15 WORDS FOR TOPIC #2
['national', 'resident', 'day', 'land', 'park', 'local', 'state', 'home', 'year', 'area', '—', '’', 'people', 'water', 'city']


THE TOP 15 WORDS FOR TOPIC #3
['come', 'cook', 'drink', 'use', 'sell', 'farmer', 'grow', 'restaurant', 'farm', 'year', 'eat', 'like', '’', '—', 'food']


THE TOP 15 WORDS FOR TOPIC #4
['think', 'like', 'change', 'researcher', 'new', 'animal', 'research', 'university', 'science', 'use', 'study', 'scientist', 'human', '’', '—']


THE TOP 15 WORDS FOR TOPIC #5
['time', 'lot', 'way', 'child', 'student', 'work', 'thing', 'want', '—', 'know', 'school', 'like', 'think', 'people', '’']


THE TOP 15 WORDS FOR T

### How to see the dominant topic in each document?
To classify a document as belonging to a particular topic, a logical approach is to see which topic has the highest contribution to that document and assign it.

In [43]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(dtm)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(npr))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
Doc0,0.75,0,0,0.01,0,0,0.04,0,0.04,0,0.05,0,0,0,0,0,0.06,0.06,0,0,0
Doc1,0.66,0,0.01,0,0,0,0.07,0,0,0,0,0,0,0.06,0,0.05,0.08,0.07,0,0,0
Doc2,0.81,0,0.01,0,0,0,0.02,0,0,0,0.09,0,0,0,0,0,0.03,0.03,0,0,0
Doc3,0.95,0,0,0,0,0.02,0,0,0,0,0,0,0,0,0,0,0,0.03,0,0,0
Doc4,0.04,0.06,0.09,0.05,0,0.15,0.2,0.01,0.08,0.02,0.07,0,0.02,0.05,0,0,0,0.05,0.02,0.08,6
Doc5,0.02,0,0.02,0.01,0,0.27,0,0,0,0.21,0,0.11,0,0.01,0.01,0,0,0.3,0.02,0,17
Doc6,0.01,0.04,0,0.01,0.12,0,0,0,0,0.12,0,0.54,0,0.01,0,0,0,0.06,0.02,0.05,11
Doc7,0,0,0.07,0.31,0,0.43,0.16,0.01,0,0,0.02,0,0,0,0,0,0,0,0,0,5
Doc8,0,0,0,0.04,0.29,0.33,0,0,0.02,0.13,0,0,0,0.11,0,0.05,0,0,0,0.03,5
Doc9,0.01,0,0.15,0.2,0.03,0.39,0,0.01,0,0,0,0,0,0,0,0.01,0,0.13,0,0.08,5


### Review topics distribution across documents

In [44]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,17,2243
1,5,1265
2,0,1041
3,10,912
4,1,881
5,19,785
6,8,754
7,2,678
8,9,603
9,6,586


In [47]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, dtm, cv, mds='tsne')
panel

//anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
17    -86.499794  -24.728388       1        1  16.212513
5     -45.483940   55.916092       2        1  12.261031
0      51.012604  -11.924231       3        1   7.963964
1     -48.353298  -39.737705       4        1   7.917775
10     23.573503  -41.621326       5        1   6.734830
2      73.035522   26.893032       6        1   5.548000
6     -79.810104   24.344036       7        1   5.420600
19    -14.545911  -21.815125       8        1   5.034449
8      33.216309  -84.719864       9        1   4.954463
12     66.676567  -56.335613      10        1   4.583606
9      93.236069  -17.338442      11        1   4.436100
4     -13.395590   26.475901      12        1   4.241825
3      47.507359   68.766045      13        1   3.995396
13     -1.097964   69.087471      14        1   3.130724
18     -9.715636 -101.535225      15        1   1.778947
11    -10.645280  -60.620472      16        1   1.594515
16    -44.301781    2.765190      17        1   1.461703
7     -55.911179  -80.576408      18        1   1.255533
14     28.624691   31.539574      19        1   0.801904
15     14.332597   -2.143838      20        1   0.672122, topic_info=      Category          Freq        Term         Total  loglift  logprob
16014  Default  22825.000000       trump  22825.000000  30.0000  30.0000
3362   Default   6407.000000     company   6407.000000  29.0000  29.0000
16112  Default  13530.000000          u.  13530.000000  28.0000  28.0000
3128   Default   7779.000000     clinton   7779.000000  27.0000  27.0000
7333   Default   8425.000000      health   8425.000000  26.0000  26.0000
...        ...           ...         ...           ...      ...      ...
16855  Topic20    353.514601         war   3629.081535   2.6737  -4.3939
3244   Topic20    190.705297  collection    593.833632   3.8666  -5.0111
17170  Topic20    266.517108       world   8473.131502   1.5433  -4.6764
1624   Topic20    207.300972        bank   1394.115920   3.0966  -4.9276
7497   Topic20    208.029684     history   3134.772264   2.2899  -4.9241

[1475 rows x 6 columns], token_table=       Topic      Freq Term
term                       
0          1  0.326346   ..
0          2  0.463616   ..
0          3  0.042940   ..
0          4  0.002493   ..
0          5  0.008172   ..
...      ...       ...  ...
17332     16  0.003680    ’
17332     17  0.001848    ’
17332     18  0.010299    ’
17332     19  0.000316    ’
17332     20  0.000142    ’

[5110 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 6, 1, 2, 11, 3, 7, 20, 9, 13, 10, 5, 4, 14, 19, 12, 17, 8, 15, 16])

### Get the top 15 keywords each topic

In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

### Attaching Discovered Topic Labels to Original Articles

In [ ]:
dtm

In [ ]:
dtm.shape

In [ ]:
len(npr)

In [ ]:
topic_results = lda_model.transform(dtm)

In [ ]:
topic_results.shape

In [ ]:
topic_results[0]

In [ ]:
topic_results[0].round(2)

In [ ]:
topic_results[0].argmax()

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [ ]:
npr.head()

In [ ]:
topic_results.argmax(axis=1)

In [ ]:
npr['Topic'] = topic_results.argmax(axis=1)

In [ ]:
npr.head(10)

## Great work!

### How to GridSearch the best LDA model?
The most important tuning parameter for LDA models is n_components (number of topics). In addition, I am going to search learning_decay (which controls the learning rate) as well.

Besides these, other possible search params could be learning_offset (downweigh early iterations. Should be > 1) and max_iter. These could be worth experimenting if you have enough computing resources.

Be warned, the grid search constructs multiple LDA models for all possible combinations of param values in the param_grid dict. So, this process can consume a lot of time and resources.

In [ ]:
from sklearn.model_selection import GridSearchCV


# Define Search Param
search_params = {'n_components': [15, 20, 25],'learning_decay': [.5, .7]}

# Init the Model
lda = LatentDirichletAllocation(max_iter=25)

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params, cv=3, verbose=2, n_jobs = -1)

# Do the Grid Search
model.fit(dtm)

How to see the best topic model and its parameters?


In [ ]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

### Compare LDA Model Performance Scores

In [ ]:
# Get Log Likelyhoods from Grid Search Output
n_topics = [15, 20, 25]
log_likelyhoods_5 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.5]
log_likelyhoods_7 = [round(gscore.mean_validation_score) for gscore in model.grid_scores_ if gscore.parameters['learning_decay']==0.7]

# Show graph
plt.figure(figsize=(12, 8))
plt.plot(n_topics, log_likelyhoods_5, label='0.5')
plt.plot(n_topics, log_likelyhoods_7, label='0.7')
plt.title("Choosing Optimal LDA Model")
plt.xlabel("Num Topics")
plt.ylabel("Log Likelyhood Scores")
plt.legend(title='Learning decay', loc='best')
plt.show()

https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/